In [1]:
import mlflow
import datetime as dt
from model1 import *
from preprocess import *
# from train import *

In [3]:
ticker = 'AAPL'
start_date = '2023-01-01'
end_date = '2024-01-01'
download_start = '2022-11-01'

In [ ]:
def save_model(self, file_path):
        with open(file_path, 'wb') as file:
            pickle.dump(self.q_table, file)

In [12]:
train(ticker, start_date, end_date, download_start, './artifacts/model.pkl')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Episode 1/249, Reward: 3679.7418212890625
Episode 2/249, Reward: 4400.117378234863
Episode 3/249, Reward: 2944.8074645996094
Episode 4/249, Reward: 1994.259536743164
Episode 5/249, Reward: 1317.198471069336
Episode 6/249, Reward: 4059.2222442626953
Episode 7/249, Reward: 2969.7888259887695
Episode 8/249, Reward: 3614.0705337524414
Episode 9/249, Reward: 3354.7288818359375
Episode 10/249, Reward: 3049.4403533935547
Episode 11/249, Reward: 1936.5672912597656
Episode 12/249, Reward: 1408.640739440918
Episode 13/249, Reward: 1497.7459869384766
Episode 14/249, Reward: 1832.7748413085938
Episode 15/249, Reward: 4070.078369140625
Episode 16/249, Reward: 2985.564712524414
Episode 17/249, Reward: 2927.3547744750977
Episode 18/249, Reward: 2886.7154998779297
Episode 19/249, Reward: 3276.879280090332
Episode 20/249, Reward: 1301.520736694336
Episode 21/249, Reward: 158.1470947265625
Episode 22/249, Reward: 3845.089340209961
Episode 23/249, Reward: 4665.7855224609375
Episode 24/249, Reward: 251.84

In [42]:
from mlflow.tracking import MlflowClient
mlflow.set_tracking_uri("sqlite:///mlflow.db")
client = MlflowClient()

prd_model_run_id = client.get_model_version_by_alias("stock-trade-action-best-model", "prd").run_id

# artifact_uri = best_performance_model.info.artifact_uri

mlflow.artifacts.download_artifacts(run_id= prd_model_run_id, dst_path = "./web_service/")

'/Users/suwarath/Documents/MLOps/StockAnalysis/web_service/'

In [38]:
artifact_uri

'/Users/suwarath/Documents/MLOps/StockAnalysis/mlruns/1/e5c6702f369b40f0b187f49c718e9ce1/artifacts'

In [14]:
artifact_uri = best_performance_model.info.artifact_uri

In [33]:
mlflow.artifacts.download_artifacts(artifact_uri = artifact_uri+"/model1/qtable.bin", dst_path = "./web_service/model/")

'/Users/suwarath/Documents/MLOps/StockAnalysis/web_service/model.bin/qtable.bin'

In [36]:
import shutil

shutil.copy("./workflow/model2.py", "./workflow/temp/model.py")

'./workflow/temp/model.py'

In [ ]:
with ("./workflow/temp/")

In [29]:
import importlib
mm = importlib.import_module(m)

ModuleNotFoundError: No module named '/Users/suwarath/Documents/MLOps/StockAnalysis/mlruns/1/e5c6702f369b40f0b187f49c718e9ce1/artifacts/model1'

In [20]:
m = mlflow.pyfunc.load_pyfunc(artifact_uri+"/model1.py")

/var/folders/1c/xy5l3bvx653590381s90cwv80000gn/T/ipykernel_32634/1168813201.py:1: FutureWarning: ``mlflow.pyfunc.load_pyfunc`` is deprecated since 1.0. This method will be removed in a future release. Use ``mlflow.pyfunc.load_model`` instead.
  m = mlflow.pyfunc.load_pyfunc(artifact_uri+"/model1.py")


MlflowException: Could not find an "MLmodel" configuration file at "/Users/suwarath/Documents/MLOps/StockAnalysis/mlruns/1/e5c6702f369b40f0b187f49c718e9ce1/artifacts/model1.py/MLmodel"

In [7]:
import numpy as np
import pandas as pd
import yfinance as yf
from ta.trend import MACD
import datetime as dt
from web_service.preprocess import *
import pickle
# m = importlib.import_module(model)

def load_and_get_action(ticker, target_date):
    date = dt.datetime.strptime(target_date, "%Y-%m-%d")
    start_date = (date - dt.timedelta(days=60)).strftime('%Y-%m-%d')
    end_date = (date + dt.timedelta(days=1)).strftime('%Y-%m-%d')
    data = yf.download(ticker, start=start_date, end=end_date).reset_index()
    data['macd'] = MACD(data['Close']).macd()
    data['obv'] = calculate_obv(data)
    target_date_data = data[data['Date'] == target_date]

    if len(target_date_data) == 0:
        msg = "Market is close"
        return msg, msg, msg

    state = np.array([
        target_date_data['Close'],
        target_date_data['Volume'],
        target_date_data['macd'],
        target_date_data['obv']
    ])
    env_start = (date - dt.timedelta(days = 425)).strftime('%Y-%m-%d')
    env_end = date.strftime('%Y-%m-%d')
    env_real_start = (date - dt.timedelta(days = 365)).strftime('%Y-%m-%d')

    env = StockTradingEnv(ticker, env_start, env_end, env_real_start)
    agent = QLearningAgent(env)
    
    with open('./models/model1/q_table.bin', 'rb') as file:
        agent.q_table = pickle.load(file)

    action = agent.choose_action(state)
    action_mapping = {0: 'Sell', 1: 'Hold', 2: 'Buy'}
    
    return action_mapping[action], state, target_date_data

In [8]:
result = load_and_get_action('AAPL', '2024-06-25')
print(f"Today's action: {result[0]}")

[*********************100%%**********************]  1 of 1 completed


ModuleNotFoundError: No module named 'model1'

In [9]:
with open('./models/model1/env.bin', 'rb') as file:
        StockTradingEnv = pickle.load(file)

ModuleNotFoundError: No module named 'model1'

In [18]:
from model_element.preprocess import *
from model_element.model import *
from flask import jsonify

ticker = 'AAPL'
target_date = '2024-01-05'
date = dt.datetime.strptime(target_date, "%Y-%m-%d")
start_date = (date - dt.timedelta(days=60)).strftime('%Y-%m-%d')
end_date = (date + dt.timedelta(days=1)).strftime('%Y-%m-%d')
data = yf.download(ticker, start=start_date, end=end_date).reset_index()
data['macd'] = MACD(data['Close']).macd()
data['obv'] = calculate_obv(data)
target_date_data = data[data['Date'] == target_date].reset_index(drop = True)

# if len(target_date_data) == 0:
#     msg = "Market is close"
    # return msg, msg, msg, msg

state = np.array([target_date_data.loc[0,i] for i in parameters])
env_end = date.strftime('%Y-%m-%d')
env_start = (date - dt.timedelta(days = 365)).strftime('%Y-%m-%d')
env = StockTradingEnv(ticker, env_start, env_end)
agent = QLearningAgent(env)

with open('./model_element/qtable.bin', 'rb') as file:
    agent.q_table = pickle.load(file)
    
action = agent.choose_action(state)
action_mapping = {0: 'Sell', 1: 'Hold', 2: 'Buy'}

parm_dict = {}
for i in parameters:
    parm_dict[i] = target_date_data.loc[0,i]
    
result = {
        'target_date': target_date,
        'action': action,
        'state': state,
        'parm_dict': parm_dict
    }

print(result)
print(jsonify(result))

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

{'target_date': '2024-01-05', 'action': 1, 'state': array([ 1.81179993e+02,  6.23033000e+07, -1.52743540e+00, -5.24859000e+07]), 'parm_dict': {'Close': 181.17999267578125, 'Volume': 62303300, 'macd': -1.5274354035035174, 'obv': -52485900}}


RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
the current application. To solve this, set up an application context
with app.app_context(). See the documentation for more information.

In [19]:
parm_dict

{'Close': 181.17999267578125,
 'Volume': 62303300,
 'macd': -1.5274354035035174,
 'obv': -52485900}

In [17]:
target_date_data.loc[0,'Close']

181.17999267578125

In [11]:
[target_date_data.loc[i] for i in parameters]

KeyError: 'Close'